쿠팡에서 데이터 긁어오기

1. 패키지 설치

In [ ]:
# requests : 온라인상의 데이터 수집
import requests
# BeautifulSoup : 
from bs4 import BeautifulSoup
# pandas : 표 형식으로 데이터를 정리
from pandas import DataFrame

2. 데이터 수집할 웹 페이지

In [ ]:
# 수집할 컨텐츠가 있는 웹 페이지 주소
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"

# 브라우저 버전정보
userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

# 접속 객체 생성
# request는 직접 header 정보 갱신이 불가능해 session 생성
session = requests.Session()

# 접속 객체에 부가정보(header) 삽입
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})

# 생성한 접속 객체를 활용해 API에 접속
r = session.get(url)

# 접속 실패 시 에러 코드와 에러 메시지 출력
if r.status_code != 200:
    msg = "[%d Error] %s 에러 발생"%(r.status_code, r.reason)
    raise Exception(msg)

# 인코딩 형식을 지정해 beautifulsoup 객체를 생성
r.encoding = 'utf-8'
# r 변수에 저장된 text 정보를 시각화, 확인
# print(r.text)

# BeautifulSoup 객체 soup를 생성
soup = BeautifulSoup(r.text)
soup

3. 필요한 내용 추출\
dataList에서 soup에 담겨지는 select의 범위를 비교해 볼 것

In [ ]:
# class가 post인 요소의 데이터 추출
# dataList = soup.select('.search-product')
product = soup.select('.search-product-wrap')
print("검색결과 수: ", len(product))

아래는 내가 짠 코드

In [ ]:
# 추출할 데이터 삽입 리스트
data = []

for p in product:

    # 상품 이름 추출
    nameEl = p.select(".name")
    if nameEl:
        name = nameEl[0].text.strip()
    else: 
        # 상품 이름이 없는 경우 처리
        name = None
    # print(name)
    

    # 가격(정가)
    basePriceEl = p.select(".base-price")
    if basePriceEl:
        basePrice = int(basePriceEl[0].text.strip().replace(",", ""))
    # .replace(",", "/")로 금액에 사용되는 ,를 공백으로 변경해 가겨의(문자열) int 변환 시 에러를 방지
    else:
        basePrice = None  # 결측치 생성(정가 없는 경우)
    # print(basePrice)


    # 할인율 추출
    discountEl = p.select(".instant-discount-rate")
    if discountEl:
        discount = int(discountEl[0].text.strip().replace("%",""))/100
        # 수치를 위해서는 숫자 값만 필요하므로 %는 삭제, 백분율을 위해 나누기 100
    else:
        discount = None # 결측치(정가 없는 경우)
    # print(discount)


    # 할인 가격 추출
    priceEl = p.select(".price-value")
    if priceEl:
        price = int(priceEl[0].text.strip().replace(",", ""))
        # 수치를 위해서는 숫자 값만 필요하므로 가격의 ,는 삭제
    else:
        price = None # 결측치(정가 없는 경우)
    # print(discount)


    # 별점 추출
    ratingEl = p.select(".rating")
    if ratingEl:
        rating = float(ratingEl[0].text.strip())
    else:
        rating = None
    # print(rating)


    # 리뷰수 rating-total-count
    reviewEl = p.select(".rating-total-count")
    if reviewEl:
        review = reviewEl[0].text.strip()
        review = int(review[1:-1])
        # int(review[1:-1])로 데이터의 괄호를 없애준다
    else:
        review = None
    # print(review)


    # # 상품 이미지는 blank 개념이 존재(현재 진도 범위를 벗어남)
    # # 상품 이미지 주소
    # imageEl = p.select(".search-product-wrap-img")
    # if imageEl:
    #     if "data-img-src" in imageEl[0].attrs:
    #         image = "https:" + imageEl[0].attrs['data-img-src']
    #     else:
    #         image = "https:" + imageEl[0].attrs['src']
    # else:
    #     image = None
    # #print(image)

    # 중고상품 판별 조건
    if not basePrice and not price:
        continue

    # 상품 하나에 대한 값들을 딕셔너리로 묶음
    item = {
        "상품명" : name,
        "가격" : basePrice,
        "할인율" : discount,
        "할인가" : price,
        "별점" : rating,
        "리뷰수" : review
    }
    data.append(item)
df = DataFrame(data)
df

크롤링은 법적으로 불법이니 코드 공부는 상관없지만 결과물은 문제 생기니 엑셀, txt 등 파일로 저장하지 말 것

In [ ]:
# # 수집한 정보를 excel 형식으로 저장
# df = DataFrame(result)
# df.to_excel("쿠팡상품정보.xlsx", index=False)
# df